# Imports

In [1]:
import pandas as pd
import numpy as np

---

In [2]:
events = pd.read_pickle("../../../../data/tp2/events_tp2_formateado.pkl")

In [3]:
events["dayofweek"] = (events["date"].dt.dayofweek).astype("int8")

---

# Armado de ventanas

In [4]:
TAMAÑO_VENTANA = 2
FECHA_INICIO = 17

In [5]:
def etiquetar_ventanas(dataset,col_fecha):
    for ventana_nro in range(1,6):
        
        fecha_inicio = FECHA_INICIO + ventana_nro
        fecha_fin = fecha_inicio + TAMAÑO_VENTANA
        
        dataset["ventana_{}".format(ventana_nro)] = (((fecha_inicio) <= dataset[col_fecha].dt.day) & ((dataset[col_fecha].dt.day) <= (fecha_fin)))
    return dataset    

In [6]:
events = etiquetar_ventanas(events,"date")

---

# Creación de features

## Creación de sets de entrenamiento

In [7]:
def crear_targets(dataframe,col_dispositivo):
    entrenamientos = {}
    for ventana_nro in range(1,6):
        ent = dataframe.loc[dataframe["ventana_{}".format(ventana_nro)]][col_dispositivo].drop_duplicates().to_frame()
        ent.set_index(col_dispositivo, inplace=True)
        entrenamientos[ventana_nro] = ent
        
    return entrenamientos

In [8]:
Xs = crear_targets(events,"device_id")

---

## Función generadora de features

In [9]:
def generar_feature_en_ventanas(dataframe, generador_feature, destinos):
    
    for ventana_nro in range(1, 6):
        
        ventana = dataframe.loc[dataframe["ventana_{}".format(ventana_nro)]]
        
        feature = generador_feature(ventana, ventana_nro)
        
        destinos[ventana_nro] = destinos[ventana_nro].merge(feature, left_index=True, right_index=True)

---

---

## Cantidad de eventos discrimando periodos

In [10]:
# Creo columnas para luego separar
events['hora'] = events['date'].dt.hour
events['dia'] = events['date'].dt.day
events.rename(columns = {"dayofweek":"dia_de_semana"},inplace = True)

def cantidad_apariciones_por_hora(dataframe, nro_ventana):
    apariciones_por_hora = dataframe.groupby(["device_id", "hora"]).agg(({"date" : "count"}))
    apariciones_por_hora = apariciones_por_hora.unstack().fillna(0).astype(np.int64)
    apariciones_por_hora.columns = ["eventos_hora_{}".format(hora) for hora in range(0,24)]
    return apariciones_por_hora

generar_feature_en_ventanas(events, cantidad_apariciones_por_hora, Xs)

In [11]:
def cantidad_apariciones_por_dia(dataframe, nro_ventana):
    dia_final = TAMAÑO_VENTANA + 2
    
    apariciones_por_dia = dataframe.groupby(["device_id", "dia"]).agg(({"date" : "count"}))
    apariciones_por_dia = apariciones_por_dia.unstack().fillna(0).astype(np.int64)
    apariciones_por_dia.columns = ["eventos_dia_{}".format(dia) for dia in range(1,dia_final)]
    return apariciones_por_dia

In [12]:
generar_feature_en_ventanas(events, cantidad_apariciones_por_dia, Xs)

---

### Cantidad de eventos con y sin wifi

In [13]:
events["sin_wifi"] = ~(events["wifi"])

In [14]:
def cant_eventos_wifi(dataframe,nro_ventana):
    return dataframe.groupby("device_id").agg({"wifi":"sum","sin_wifi":"sum"}).rename(columns={"wifi": "cant_eventos_wifi",
                                                                                              "sin_wifi":"cant_eventos_sin_wifi"})
    

In [15]:
generar_feature_en_ventanas(events,cant_eventos_wifi, Xs)

---

---

# Guardado de los sets de entrenamiento

In [16]:
for ventana_nro in range(1, 6):
    Xs[ventana_nro].to_pickle("../../features/events_v_{}.pkl".format(ventana_nro))
    Xs[ventana_nro].to_csv("../../features/events_v_{}.csv".format(ventana_nro))

---